### 實驗六

- 以文章中出現 object 的數量為分數，因為每篇文章的 object string 都不同，就無法設定共同的分母了。
- 例如: 文章 A 寫了 /proc/net/dev 3次，就要計算3分。(對嗎?)
- 例如: 文章 B 寫了 /var/run/gcc.pid 和 /var/run/gcc.sh 都符合 /var/run/.* 這一個 regex，計算2分。
- 例外: 因為 memory 和 network 有經過 sink node，我們只會計算成1次出現。

In [1]:
import re
import os
from os import listdir
from os.path import isfile, join
from pathlib import Path
import pandas as pd
import json
import pickle
from typing import Dict, List, get_type_hints
from IPython.display import display

''' 自己寫的模組 '''
# from cc_regex_script import RegexMatchResult, RegexMaster 
# from cc_nlp_script import find_verb_of_vocab, OperationMode, OperationEvaluator
# from ASG import AttackGraph, Node, Edge, FileTable #, build
# import Utility

' 自己寫的模組 '

In [2]:
# 舊的樣本 asg_regex = ['.*/selinux', '/.*bin/sed', '/dev/urandom', '/etc/init\\.d/.*', '/etc/rc.*\\.d/.*', '/etc/rc\\.local', '/proc/net/dev', '/proc/self', '/proc/stat', '/sys/devices/.*', '/etc/sed[0-9a-zA-Z]{6}', 'Permission', 'Permission:[0-9]{3}', '^GID.*', '^PID.*', '^UID.*', '^eth.*', '^sed$', '^sh$', '^uname$',]
asg_regex = ['.*/selinux', '/.*bin/rm', '/.*bin/sed', '/dev/urandom', '/etc/host\\.conf', '/etc/hosts', '/etc/init\\.d/.*', '/etc/nsswitch\\.conf', '/etc/rc.*\\.d/.*', '/etc/rc\\.local', '/etc/resolv\\.conf', '/etc/sed[0-9a-zA-Z]{6}', '/proc/net/dev', '/proc/self', '/proc/stat', '/sys/devices/.*', 'Permission', 'Permission:[0-9]{3}', '^GID.*', '^PID.*', '^UID.*', '^eth.*', '^rm$', '^sed$', '^sh$', '^uname$']

print(len(asg_regex))

cntone_regex = ['0x[0-9a-zA-Z]{1,16}', '\\d{1,3}\\.\\d{1,3}\\.\\d{1,3}\\.\\d{1,3}', '\\d{1,3}\\.\\d{1,3}\\.\\d{1,3}\\.\\d{1,3}:\\d+', ]
# 'port \\d+' 先移除

26


In [3]:
family = 'Dofloo'
# report_fir_path = '../../C parse report/sentence csvs/'
report_fir_path = '../../C parse report/'
# family, samplename, outputfolder = 'Xorddos', '???', './result'
outputfolder = './result'

def get_all_filenames(dir: str='./') -> list:
    ''' traverse root directory, and list directories as dirs and files as files. Return filenames in rootdir. '''
    files = [f for f in listdir(dir) if isfile(join(dir, f))]
    files.sort()
    return files

xor_report_names = [f for f in get_all_filenames(report_fir_path) if f.startswith(family)] # 只選擇這個家族的報告
# xor_report_dfs = [pd.read_csv(f"{report_fir_path}{f}") for f in xor_report_names]
xor_report_txts = []
for f in xor_report_names:
    with open(f"{report_fir_path}{f}", 'r', encoding='utf-8') as f:
        content = f.read()
        xor_report_txts.append(content)
xor_report_names

['Dofloo-BleepingComputer.txt',
 'Dofloo-Securityaffairs.txt',
 'Dofloo-SyscallParty.txt',
 'Dofloo-Trendmicro.txt']

In [4]:
txt = xor_report_txts[2]

def get_metric(txt:str, reportname:str):
    # record appearance of regex
    report_tf = {}
    for pattern in asg_regex + cntone_regex:
        cnt = len(re.findall(pattern, txt))
        report_tf[pattern] = cnt

    # calculate the metrics
    score = 0
    score_sys = 0
    for pattern, cnt in report_tf.items():
        # print(pattern)
        if pattern in cntone_regex and cnt:
            score += 1
        else:
            score += cnt
        if pattern.find('/etc') >= 0:
            if pattern in cntone_regex:
                continue
            score_sys += cnt
    print(f"{reportname:30} score: {score:2}, score_sys: {score_sys}")
    return report_tf

results = []
for txt, name in zip(xor_report_txts, xor_report_names):
    report_tf = get_metric(txt, name)
    results.append(report_tf)

Dofloo-BleepingComputer.txt    score:  0, score_sys: 0
Dofloo-Securityaffairs.txt     score:  4, score_sys: 2
Dofloo-SyscallParty.txt        score:  9, score_sys: 5
Dofloo-Trendmicro.txt          score: 15, score_sys: 9


In [5]:
results[0]

{'.*/selinux': 0,
 '/.*bin/rm': 0,
 '/.*bin/sed': 0,
 '/dev/urandom': 0,
 '/etc/host\\.conf': 0,
 '/etc/hosts': 0,
 '/etc/init\\.d/.*': 0,
 '/etc/nsswitch\\.conf': 0,
 '/etc/rc.*\\.d/.*': 0,
 '/etc/rc\\.local': 0,
 '/etc/resolv\\.conf': 0,
 '/etc/sed[0-9a-zA-Z]{6}': 0,
 '/proc/net/dev': 0,
 '/proc/self': 0,
 '/proc/stat': 0,
 '/sys/devices/.*': 0,
 'Permission': 0,
 'Permission:[0-9]{3}': 0,
 '^GID.*': 0,
 '^PID.*': 0,
 '^UID.*': 0,
 '^eth.*': 0,
 '^rm$': 0,
 '^sed$': 0,
 '^sh$': 0,
 '^uname$': 0,
 '0x[0-9a-zA-Z]{1,16}': 0,
 '\\d{1,3}\\.\\d{1,3}\\.\\d{1,3}\\.\\d{1,3}': 0,
 '\\d{1,3}\\.\\d{1,3}\\.\\d{1,3}\\.\\d{1,3}:\\d+': 0}

In [6]:
results[1]

{'.*/selinux': 0,
 '/.*bin/rm': 0,
 '/.*bin/sed': 0,
 '/dev/urandom': 0,
 '/etc/host\\.conf': 0,
 '/etc/hosts': 0,
 '/etc/init\\.d/.*': 0,
 '/etc/nsswitch\\.conf': 0,
 '/etc/rc.*\\.d/.*': 1,
 '/etc/rc\\.local': 1,
 '/etc/resolv\\.conf': 0,
 '/etc/sed[0-9a-zA-Z]{6}': 0,
 '/proc/net/dev': 0,
 '/proc/self': 0,
 '/proc/stat': 0,
 '/sys/devices/.*': 0,
 'Permission': 0,
 'Permission:[0-9]{3}': 0,
 '^GID.*': 0,
 '^PID.*': 0,
 '^UID.*': 0,
 '^eth.*': 0,
 '^rm$': 0,
 '^sed$': 0,
 '^sh$': 0,
 '^uname$': 0,
 '0x[0-9a-zA-Z]{1,16}': 0,
 '\\d{1,3}\\.\\d{1,3}\\.\\d{1,3}\\.\\d{1,3}': 1,
 '\\d{1,3}\\.\\d{1,3}\\.\\d{1,3}\\.\\d{1,3}:\\d+': 1}

In [7]:
results[2]

{'.*/selinux': 0,
 '/.*bin/rm': 0,
 '/.*bin/sed': 0,
 '/dev/urandom': 0,
 '/etc/host\\.conf': 0,
 '/etc/hosts': 0,
 '/etc/init\\.d/.*': 1,
 '/etc/nsswitch\\.conf': 0,
 '/etc/rc.*\\.d/.*': 1,
 '/etc/rc\\.local': 3,
 '/etc/resolv\\.conf': 0,
 '/etc/sed[0-9a-zA-Z]{6}': 0,
 '/proc/net/dev': 1,
 '/proc/self': 1,
 '/proc/stat': 0,
 '/sys/devices/.*': 0,
 'Permission': 0,
 'Permission:[0-9]{3}': 0,
 '^GID.*': 0,
 '^PID.*': 0,
 '^UID.*': 0,
 '^eth.*': 0,
 '^rm$': 0,
 '^sed$': 0,
 '^sh$': 0,
 '^uname$': 0,
 '0x[0-9a-zA-Z]{1,16}': 20,
 '\\d{1,3}\\.\\d{1,3}\\.\\d{1,3}\\.\\d{1,3}': 1,
 '\\d{1,3}\\.\\d{1,3}\\.\\d{1,3}\\.\\d{1,3}:\\d+': 0}

In [8]:
results[3]

{'.*/selinux': 0,
 '/.*bin/rm': 0,
 '/.*bin/sed': 0,
 '/dev/urandom': 0,
 '/etc/host\\.conf': 1,
 '/etc/hosts': 0,
 '/etc/init\\.d/.*': 1,
 '/etc/nsswitch\\.conf': 1,
 '/etc/rc.*\\.d/.*': 1,
 '/etc/rc\\.local': 4,
 '/etc/resolv\\.conf': 1,
 '/etc/sed[0-9a-zA-Z]{6}': 0,
 '/proc/net/dev': 1,
 '/proc/self': 4,
 '/proc/stat': 1,
 '/sys/devices/.*': 0,
 'Permission': 0,
 'Permission:[0-9]{3}': 0,
 '^GID.*': 0,
 '^PID.*': 0,
 '^UID.*': 0,
 '^eth.*': 0,
 '^rm$': 0,
 '^sed$': 0,
 '^sh$': 0,
 '^uname$': 0,
 '0x[0-9a-zA-Z]{1,16}': 0,
 '\\d{1,3}\\.\\d{1,3}\\.\\d{1,3}\\.\\d{1,3}': 0,
 '\\d{1,3}\\.\\d{1,3}\\.\\d{1,3}\\.\\d{1,3}:\\d+': 0}